In [47]:
import requests
import re
from bs4 import BeautifulSoup
import string
import pandas as pd
from pathlib import Path
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from pyshadow.main import Shadow
import pandas as pd
import csv, os
from selenium.common.exceptions import NoSuchElementException
import time

In [48]:
col=['AU1_r','AU2_r','AU3_r','AU4_r','AU5_r','TI_r','TY']
for i in range(1,11):
    col.append('AU'+str(i))
col = col + ["TI","JO","VL","IS","SN","DO","SP","EP"]

In [49]:
def Download_Citation(Volume, Issue):
    driver = webdriver.Chrome(ChromeDriverManager().install())
    #driver.execute_script("window.open('https://onlinelibrary.wiley.com/toc/1534875x/"+str(Volume)+"/"+str(Volume)+"/"+str(Issue)+"/');")
    driver.get("https://onlinelibrary.wiley.com/toc/1534875x/"+str(Volume)+"/"+str(Volume)+"/"+str(Issue))
    driver.find_element(By.CSS_SELECTOR, "a[class='bulk-download-option'] span").click()
    Research_Articles = driver.find_elements(By.CLASS_NAME,'checkbox--primary')
    for a in Research_Articles[1:]:
        a.click()
    driver.find_element(By.CSS_SELECTOR, ".bulk-wizard.exportCitationsForm").click()
    driver.find_element(By.CSS_SELECTOR, ".export-citations__submit").click()
    time.sleep(5)
    driver.quit()

In [54]:
def Find_Authors(html_name): 
    All = []
    with open('/Users/langchen777/Desktop/NewDirectionsforEvaluations/New Directions for Evaluations/HTML/'+html_name+'.html', 'r', encoding = 'utf-8') as f:
        soup = BeautifulSoup(f.read())
        soup = BeautifulSoup(f.read())
        content = soup.findAll("li")
    for li in content:
        authors = li.findAll("span", class_="author")
        line = []
        for x in authors:
            line.append(x.contents[0])
        while(len(line)<5):
            line.append("")
        while(len(line)>5):
            line.pop()
        try:
            title = li.find("span", class_="articleTitle").text
        except AttributeError:
            pass
        try:
            title = li.find("span", class_="chapterTitle").text
        except AttributeError:
            pass
        try:
            title = li.find("span", class_="bookTitle").text
        except AttributeError:
            pass
        try:
            title = li.find("span", class_="otherTitle").text
        except AttributeError:
            pass
        line.append(title)
        All.append(line)
        print(All)
    return All

In [55]:
def Read_Citation(issue):
    df = pd.DataFrame(columns=col)
    file = open('/Users/langchen777/Downloads/pericles_exported_citations.txt', 'r')
    Lines = file.readlines()
    count = 0
    for line in Lines:
        start = line[:2]
        if start == "TY":
            count = count + 1
            #Ref = Ref()
            row = []
            row.append(line.split("-",1)[1].strip())
        elif start == "AU":
            row.append(line.split("-",1)[1].strip())
        elif start == "TI":
            while(len(row)<11):
                row.append("")
            while(len(row)>11):
                row.pop()
            row.append(line.split("-",1)[1].strip())
        elif start in ["JO","VL","IS","SN","DO","SP","EP"]:
            row.append(line.split("-",1)[1].strip())
        elif start == "ER":
            try:
                for r in Find_Authors(str(issue)+str(count)):
                    combine = r + row
                    print(combine)
                    a_series = pd.Series(combine, index = col)
                    df = df.append(a_series, ignore_index=True)
            except FileNotFoundError:
                pass
            else:
                r = ["","","","","",""]
                combine = r + row
                print(combine)
    #os.remove("/Users/langchen777/Downloads/pericles_exported_citations.txt")
    return df

In [56]:
def Return_Issue(i):
    if i in range(1,5):
        volume = 1978
    else:
        volume = 1979 + (i-1)//4
    Download_Citation(volume, i)
    df=Read_Citation(volume,i)
    return df

In [57]:
Read_Citation(84)

['', '', '', '', '', '', 'JOUR', 'Gay, Geri', 'Bennington, Tammy L.', '', '', '', '', '', '', '', '', 'Reflective evaluation in a “technologically textured” world: An activity theory approach', 'New Directions for Evaluation', '1999', '84', '1097-6736', 'https://doi.org/10.1002/ev.1150', '3', '21']
['', '', '', '', '', '', 'JOUR', 'Watt, James H.', '', '', '', '', '', '', '', '', '', 'Internet systems for evaluation research', 'New Directions for Evaluation', '1999', '84', '1097-6736', 'https://doi.org/10.1002/ev.1151', '23', '43']
['', '', '', '', '', '', 'JOUR', 'Rieger, Robert H.', 'Sturgill, Amanda', '', '', '', '', '', '', '', '', 'Evaluating on-line environments: Tools for observing users and gathering feedback', 'New Directions for Evaluation', '1999', '84', '1097-6736', 'https://doi.org/10.1002/ev.1152', '45', '58']
['', '', '', '', '', '', 'JOUR', 'Bennington, Tammy L.', 'Gay, Geri', 'Jones, Michael L. W.', '', '', '', '', '', '', '', 'Using multimedia records to support mixed

,AU1_r,AU2_r,AU3_r,AU4_r,AU5_r,TI_r,TY,AU1,AU2,AU3,...,AU9,AU10,TI,JO,VL,IS,SN,DO,SP,EP


In [27]:
df

,AU1_r,AU2_r,AU3_r,AU4_r,AU5_r,TI_r,TY,AU1,AU2,AU3,...,AU9,AU10,TI,JO,VL,IS,SN,DO,SP,EP
